In [ ]:
import pandas as pd
import os
from glob import glob
import time

start_time = time.time()

dirpath = '/home/bioinfo/Nilesh/HRRdb_Samples/HRR_G+_VCFs_Elucidata/Germline/FE_merged/'
savepath = '/home/bioinfo/Nilesh/HRRdb_Samples/HRR_G+_VCFs_Elucidata/Germline/G_HRR/'
hrr_genes = pd.read_csv("/home/bioinfo/git/HRRdb/HRR_genes.txt", sep='/t')
chunk_size = 100  

file_list = glob(os.path.join(dirpath, "*.csv"))
chunked_list = [file_list[i:i+chunk_size] for i in range(0, len(file_list), chunk_size)]

for keys, chunk in enumerate(chunked_list):
    chunk_dir = os.path.join(savepath, str(keys))
    os.mkdir(chunk_dir)
    for file in chunk:
        os.system(f'cp {file} {chunk_dir}')
    
for keys, chunk in enumerate(chunked_list):
    chunk_dir = os.path.join(savepath, str(keys))
    os.chdir(chunk_dir)
    key_files = sorted(glob("*.csv"))
    samples = pd.concat((pd.read_csv(file, sep=',') for file in key_files), ignore_index=True)
    hrr_df = pd.merge(samples, hrr_genes, how="inner", left_on='Ref.Gene', right_on='HRR_Genes')
    hrr_df.to_csv(f'{keys}_hrr_df.csv', index=False)
    os.chdir(savepath)

os.system("notify-send 'HRD File Processor' 'Process Finished'")    
os.system("telegram-send 'HRD File Processor finished'")
